<a href="https://colab.research.google.com/github/semishen/ML100Days/blob/master/Day_025_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [77]:
# #只取類別值 (object) 型欄位, 存於 object_features 中
# object_features = []
# for dtype, feature in zip(df.dtypes, df.columns):
#     if dtype == 'object':
#         object_features.append(feature)
# print(f'{len(object_features)} Numeric Features : {object_features}\n')

# # 只留類別型欄位
# df = df[object_features]
df_obj = df.loc[:, (df.dtypes == 'object')]
df_obj = df_obj.fillna('None')
train_num = train_Y.shape[0]
df_obj.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [78]:
df_obj_target = pd.concat([df_obj[:train_num], train_Y],axis = 1) # pull target back for mean calculation
for col in df_obj.columns:
  df_tmp = df_obj_target.groupby(col)['Survived'].mean().reset_index() # calculate target mean grouped by col
  new_col_name = str(col) + '_mean' # new col name
  df_tmp.columns = [col, new_col_name] # change col name
  df_obj_target =  pd.merge(df_obj_target, df_tmp, on= col, how='left') # merge 2 df
  df_obj_target = df_obj_target.drop(col, axis=1) # drop old col
df_obj_mean_encoder = df_obj_target.drop('Survived', axis= 1)
df_obj_mean_encoder.head()

,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
0,0,0.188908,0.0,0.299854,0.336957
1,1,0.742038,1.0,1.000000,0.553571
2,1,0.742038,1.0,0.299854,0.336957
3,1,0.742038,0.5,0.500000,0.336957
4,0,0.188908,0.0,0.299854,0.336957


# 作業2
### Q: 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?
### A: 均值編碼的 mean of cross_val_score = 1.0，雖然分數上較標籤編碼 mean of cross_val_score = 0.78 表現好，但實屬 overfitting，不應該每個 feature 都做均值編碼

In [83]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_obj_label = pd.DataFrame()
for col in df_obj.columns:
  df_obj_label[col] = LabelEncoder().fit_transform(df_obj[col])
train = df_obj_label[:train_num]
print('shape: ', train.shape)

estimator = LogisticRegression()
start = time.time()
score = cross_val_score(estimator, train, train_Y, cv=5).mean()
print('score: ', score)
print('time:', time.time() - start)

shape:  (891, 5)
score:  0.7800138095537004
time: 0.12130928039550781


In [85]:
# 均值編碼 + 邏輯斯迴歸
train = df_obj_mean_encoder[:train_num]
print('shape: ', train.shape)

estimator = LogisticRegression()
start = time.time()
score = cross_val_score(estimator, train, train_Y, cv=5).mean()
print('score: ', score)
print('time:', time.time() - start)

shape:  (891, 5)
score:  1.0
time: 0.05266261100769043
